In [70]:
import pandas as pd
import requests
import json
import nltk
import contractions

from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
import keys
#from keys import subscription_key, text_analytics_base_url, bing_spell_check_key, spell_check_url 

In [36]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/azureuser/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [69]:
# !pip install contractions

In [114]:
# !pip install --upgrade azure-cognitiveservices-language-textanalytics
# from azure.cognitiveservices.language.textanalytics import TextAnalyticsClient
# from msrest.authentication import CognitiveServicesCredentials

In [351]:
sample_tweets = pd.DataFrame.from_csv(
    "sample_tweets.csv",
    index_col=None
)

In [352]:
sample_tweets["id"] = sample_tweets.index
sample_tweets.rename(columns={"Body text": "text"}, inplace=True)
#sample_tweets.text = sample_tweets.text.str.lower()
sample_tweets.head(2)

,Down ID,Posted time,text,Point latitiude,Point longitude,Bio location,Image url,id
0,"""tag:search.twitter.com,2005:1109081183105835008""",22/03/2019 13:15,"""Primeape was recently spotted trying to make ...",NaN,NaN,NaN,"""https://pbs.twimg.com/profile_images/76017773...",0
1,"""tag:search.twitter.com,2005:1109080961277407234""",22/03/2019 13:14,"""@gianninewbon Meet me at London Bridge at hal...",NaN,NaN,"""Derbados""","""https://pbs.twimg.com/profile_images/10569615...",1


In [91]:
sample_tweets.count()

Down ID            44816
Posted time        44816
text               44816
Point latitiude      701
Point longitude      701
Bio location       31946
Image url          42978
id                 44816
dtype: int64

In [64]:
def spell_check(sentence):
    data = {'text': sentence}
    params = {
        'mkt':'en-us',
        'mode':'proof'
        }
    headers = {
        'Content-Type': 'application/x-www-form-urlencoded',
        'Ocp-Apim-Subscription-Key': bing_spell_check_key,
        }
    response = requests.post(spell_check_url, headers=headers, params=params, data=data)
    json_response = response.json()

    for token in json_response["flaggedTokens"]:
        sentence = sentence.replace(str(token["token"]), token["suggestions"][0]["suggestion"])
    
    return sentence

In [310]:
# stop_words = set(stopwords.words('english'))
tokenizer = TweetTokenizer(strip_handles=True, reduce_len=True)
def process_tweet(tweet):
    normalized_tweet = contractions.fix(spell_check(tweet).lower().strip())
    tokenized = tokenizer.tokenize(normalized_tweet)

    # tokenized = nltk.word_tokenize(normalized_tweet)
    # non_stop_words = [word for word in tokenized if (word not in stop_words)]
    
    isascii = lambda s: len(s) == len(s.encode())
    ascii_words = [word for word in tokenized[1:-1] if isascii(word)]
    normalized_tweet = " ".join(ascii_words)
    
    return normalized_tweet
    

In [311]:
subset_tweets = sample_tweets[["id", "text"]].head(1000)
subset_tweets["text"] = subset_tweets["text"].apply(process_tweet)
documents_str = subset_tweets.to_json(orient = "records")
documents_json = json.loads(json.dumps({"documents" : json.loads(documents_str)}))

In [334]:
subset_tweets.head()

,id,text
0,0,primeape was recently spotted trying to make f...
1,1,meet me at london bridge at half 3
2,2,"unreal city , under the brown fog of a winter ..."
3,3,there is plenty to explore in london ! sightse...
4,4,cycling near london bridge ? stop off for a ch...


In [335]:
documents_json["documents"][0:5]

[{'id': 0,
  'text': 'primeape was recently spotted trying to make friends in front of tower bridge .'},
 {'id': 1, 'text': 'meet me at london bridge at half 3'},
 {'id': 2,
  'text': 'unreal city , under the brown fog of a winter dawn , a crowd flowed over london bridge , so many , i had not thought death had undone so many .'},
 {'id': 3,
  'text': 'there is plenty to explore in london ! sightsee on the iconic london open-top double decker buses , explore tower bridge , visit the london eye and take a selfie in front of buckingham palace . click below to see more travel tips !'},
 {'id': 4,
  'text': 'cycling near london bridge ? stop off for a chat , a cider and maybe even something special to take home . #cider bar #londonbars #tap room #londoncycling #cycle meets #cycle stops https://t.co/w8gikpgcx1'}]

In [225]:
keyphrase_url = text_analytics_base_url + "keyPhrases"

headers   = {"Ocp-Apim-Subscription-Key": subscription_key}
response  = requests.post(keyphrase_url, headers=headers, json=documents_json)
key_phrases = response.json()

key_phrases_df = pd.DataFrame.from_dict(key_phrases['documents'])
key_phrases_df.head()

In [228]:
(key_phrases_df["keyPhrases"].apply(lambda x: len(x)).mean(),
key_phrases_df["keyPhrases"].apply(lambda x: len(x)).max(),
key_phrases_df["keyPhrases"].apply(lambda x: len(x)).min())

(5.111, 14, 1)

In [261]:
unique_key_phrases = set(sum(key_phrases_df["keyPhrases"].values, []))

key_phrases_category = {}
for key in unique_key_phrases:
    key_phrases_category[key] = []

def map_categories(row):
    for phrase in row["keyPhrases"]:
        key_phrases_category[phrase].append(row["id"])
    return None

key_phrases_df.apply(map_categories, axis=1)

key_phrases_category_count={}
for key, value in key_phrases_category.items():
    key_phrases_category_count[key] = len(value)

from collections import OrderedDict
key_phrases_category_count = OrderedDict(
                               sorted(key_phrases_category_count.items(), 
                                      key=lambda kv: kv[1], 
                                      reverse=True)
                            )

print(len(key_phrases_category_count))
# print(key_phrases_category_count)

2550


In [394]:
key_phrases_category_count

OrderedDict([('london bridge', 419),
             ('days', 121),
             ('london bridge terrorist attack', 119),
             ('islamophobic', 118),
             ('interior department spokeswoman', 118),
             ('anti-immigrant rhetoric', 116),
             ('podcast episode', 112),
             ('tower bridge', 108),
             ('eb', 32),
             ('west sacramento', 32),
             ('london', 31),
             ('people', 27),
             ('time', 25),
             ('minutes', 24),
             ('london bridge station', 22),
             ('tower bridge gtwy', 19),
             ('fair lady', 17),
             ('day', 16),
             ('uk', 15),
             ('inconvenience', 15),
             ('morning', 15),
             ('heavy traffic jam', 15),
             ('battersea', 15),
             ('london bridge attack', 14),
             ('train', 14),
             ('thanks', 13),
             ('harbor blvd', 13),
             ('trains', 13),
             ('bridge 

In [366]:
# stop_words = set(stopwords.words('english'))
tokenizer = TweetTokenizer(strip_handles=True, reduce_len=True)
def process_tweet(tweet):
    normalized_tweet = contractions.fix(spell_check(tweet).strip())
    tokenized = tokenizer.tokenize(normalized_tweet)
    
    isascii = lambda s: len(s) == len(s.encode())
    ascii_words = [word for word in tokenized[1:-1] if isascii(word)]
    normalized_tweet = " ".join(ascii_words)
    
    return normalized_tweet

In [367]:
subset_tweets_entities = sample_tweets[["id", "text"]].head(1000)
subset_tweets_entities["text"] = subset_tweets_entities["text"].apply(process_tweet)

documents_str = subset_tweets_entities.to_json(orient = "records")
documents_json = json.loads(json.dumps({"documents" : json.loads(documents_str)}))

In [368]:
entities_url = text_analytics_base_url + "entities"

response  = requests.post(entities_url, headers=headers, json=documents_json)
entities = response.json()
entities_df = pd.DataFrame.from_dict(entities['documents'])

def process_entities(row):
    entities = []
    for entity in row["entities"]:
        entities.append(entity["name"])
    return entities

entities_df["entities_processed"] = entities_df.apply(process_entities, axis=1)


In [369]:
def no_entities(entities):
    if len(entities) == 0:
        return 1
not_covered =  entities_df["entities_processed"].apply(no_entities).sum()

print("Number of posts not covered by entities: {} or {} %".format(not_covered, 
                                                                   (not_covered/1000.0*100)))

Number of posts not covered by entities: 0 or 0.0 %


In [370]:
(
  entities_df["entities_processed"].apply(lambda x: len(x)).mean(),
  entities_df["entities_processed"].apply(lambda x: len(x)).max(),
  entities_df["entities_processed"].apply(lambda x: len(x)).min()
)

(3.723, 18, 1)

In [371]:
unique_entities = set(sum(entities_df["entities_processed"].values, []))

entities_category = {}
for key in unique_entities:
    entities_category[key] = []

def map_entities(row):
    for phrase in row["entities_processed"]:
        entities_category[phrase].append(row["id"])
    return None

entities_df.apply(map_entities, axis=1)

entities_category_count={}
for key, value in entities_category.items():
    entities_category_count[key] = len(value)

from collections import OrderedDict
entities_category_count = OrderedDict(
                               sorted(entities_category_count.items(), 
                                      key=lambda kv: kv[1], 
                                      reverse=True)
                            )

print(len(entities_category_count))

1474


In [391]:
entities_category_count

OrderedDict([('London Bridge', 254),
             ('London Bridge station', 210),
             ('Tower Bridge', 187),
             ('2017 London Bridge attack', 156),
             ('Islamophobia', 119),
             ('United States Department of the Interior', 118),
             ('London', 107),
             ('London Bridge Is Falling Down', 62),
             ('London Bridge (Fergie song)', 52),
             ('West Sacramento, California', 32),
             ('Tower Bridge (California)', 29),
             ('United Kingdom', 26),
             ('London Victoria station', 24),
             ('EB Games', 23),
             ('U.S. Route 50 in Maryland', 20),
             ('Fergie (singer)', 19),
             ('Battersea', 15),
             ('Waterloo Bridge', 15),
             ('River Thames', 13),
             ('Heavy Traffic (album)', 13),
             ('The New York Times', 13),
             ('Harbor Boulevard', 13),
             ('Outer space', 12),
             ('Manchester', 12),
       

In [387]:
pd.DataFrame(list(entities_category_count))

,0
0,London Bridge
1,London Bridge station
2,Tower Bridge
3,2017 London Bridge attack
4,Islamophobia
5,United States Department of the Interior
6,London
7,London Bridge Is Falling Down
8,London Bridge (Fergie song)
9,"West Sacramento, California"


In [377]:
entities_df.head()

,entities,id,entities_processed
0,"[{'name': 'Tower Bridge', 'wikipediaUrl': 'htt...",0,[Tower Bridge]
1,"[{'name': 'London Bridge', 'wikipediaUrl': 'ht...",1,[London Bridge]
2,"[{'name': 'London Bridge', 'wikipediaUrl': 'ht...",2,"[London Bridge, Unreal (1998 video game), Unde..."
3,"[{'name': 'Buckingham Palace', 'wikipediaUrl':...",3,"[Buckingham Palace, Tower Bridge, London Eye, ..."
4,"[{'name': 'London Bridge', 'wikipediaUrl': 'ht...",4,"[London Bridge, Cycling, London]"


In [378]:
key_phrases_df.head()

,id,keyPhrases
0,0,"[friends, tower bridge, primeape]"
1,1,[london bridge]
2,2,"[brown fog, winter dawn, crowd, london bridge,..."
3,3,"[london eye, iconic london open, double decker..."
4,4,"[cycle, cider bar, room, londonbars, londoncyc..."


In [379]:
subset_tweets.head()

,id,text
0,0,primeape was recently spotted trying to make f...
1,1,meet me at london bridge at half 3
2,2,"unreal city , under the brown fog of a winter ..."
3,3,there is plenty to explore in london ! sightse...
4,4,cycling near london bridge ? stop off for a ch...


In [380]:
metadata_entities = pd.merge(key_phrases_df[["id", "keyPhrases"]], entities_df[["id", "entities_processed"]], on="id", how="inner")

metadata_entities.id = metadata_entities.id.astype(int)
metadata_entities.keyPhrases = metadata_entities.keyPhrases.astype(str)
metadata_entities.entities_processed = metadata_entities.entities_processed.astype(str)

subset_tweets.id = subset_tweets.id.astype(int)
subset_tweets.text = subset_tweets.text.astype(str)


In [381]:
tweets_meta_data_entities = pd.merge(metadata_entities, subset_tweets, on="id", how="inner")

In [382]:
pd.merge(tweets_meta_data_entities, 
         sample_tweets[["id","text"]].head(1000),
         on="id",
         how="inner")

,id,keyPhrases,entities_processed,text_x,text_y
0,0,"['friends', 'tower bridge', 'primeape']",['Tower Bridge'],primeape was recently spotted trying to make f...,"""Primeape was recently spotted trying to make ..."
1,1,['london bridge'],['London Bridge'],meet me at london bridge at half 3,"""@gianninewbon Meet me at London Bridge at hal..."
2,2,"['brown fog', 'winter dawn', 'crowd', 'london ...","['London Bridge', 'Unreal (1998 video game)', ...","unreal city , under the brown fog of a winter ...","""Unreal City, Under the brown fog of a winter ..."
3,3,"['london eye', 'iconic london open', 'double d...","['Buckingham Palace', 'Tower Bridge', 'London ...",there is plenty to explore in london ! sightse...,"""@hu9654a there’s plenty to explore in London!..."
4,4,"['cycle', 'cider bar', 'room', 'londonbars', '...","['London Bridge', 'Cycling', 'London']",cycling near london bridge ? stop off for a ch...,"""Cycling near London Bridge? Stop off for a ch..."
5,5,"['tower bridge', 'brick', 'amazing arch design...","['Tower Bridge', 'Instagram']",amazing arch design using our tower bridge #br...,"""Amazing arch design using our Tower Bridge #b..."
6,6,"[""china's copycat version of london's tower br...","['Justice League', 'Tower Bridge', 'Bishop of ...",china's copycat version of london's tower brid...,"""China's copycat version of London's Tower Bri..."
7,7,"['prayer i', 'new zealand mosque attacks', 'lo...","['2017 London Bridge attack', 'New Zealand', '...",as we prepare to gather this afternoon for the...,"""As we prepare to gather @pborocathedral this ..."
8,8,"['prayer i', 'new zealand mosque attacks', 'lo...","['2017 London Bridge attack', 'New Zealand', '...",as we prepare to gather this afternoon for the...,"""As we prepare to gather @pborocathedral this ..."
9,9,"['fake tower bridge', 'renovation']","['Tower Bridge', 'Fake (manga)']",fake tower bridge is ridiculed despite 2.2 mil...,"""Fake Tower Bridge is ridiculed despite £2.2 m..."


In [383]:
tweets_meta_data_entities.to_csv("tweet_categories.csv")